In [1]:
from typing import Annotated, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
)

from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

In [2]:
model = ChatOpenAI(model = 'gpt-5-2025-08-07')

In [3]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

In [4]:
generate_prompt = SystemMessage(
    "당신은 훌륭한 자기소개서를 작성하는 임무를 가진 자기소개서 작성 어시스턴트입니다."
    "사용자의 요청에 맞춰 최상의 자기소개서를 작성하세요"
    "사용자가 주제 및 맥락을 제공하면, 이전 시도에 대한 수정 버전을 응답하세요."
)

In [5]:
reflection_prompt = SystemMessage(
    "당신은 자기소개서를 읽어보는 인사 담당자입니다., 사용자의 자기소개서를 읽고 비평이나 수정사항을 생성하세요"
    "변경해야하는 내용, 스타일, 깊이등 같이 구체적인 요구사항을 포함한 자세한 내용을 제공하세요 "
)

In [6]:
builder = StateGraph(State)

In [7]:
def generate(state: State) -> State:
    answer = model.invoke([generate_prompt] + state['messages'])
    return {'messages' : [answer]}

In [8]:
def reflect(state : State) -> State:
    cls_map = {AIMessage :  HumanMessage, HumanMessage : AIMessage}
    translated = [reflection_prompt, state['messages'][0]] +[cls_map[msg.__class__](content=msg.content)   for msg in state['messages'][1:]]
    answer = model.invoke(translated)
    return {'messages' : [HumanMessage(content=answer.content)]}

In [ ]:
def should_continue(state : State) : 
    if len(state['messages']) > 6 :
        return END
    else:
        return 'reflect'

In [10]:
builder = StateGraph(State)
builder.add_node('generate', generate)
builder.add_node('reflect', reflect)
builder.add_edge(START, 'generate')
builder.add_conditional_edges('generate', should_continue)
builder.add_edge('reflect', 'generate')
graph = builder.compile()

In [11]:
graph.get_graph().draw_mermaid_png(output_file_path="./자소서2.png")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xec\x00\x00\x01\x08\x08\x02\x00\x00\x00\xd3\x13\x8fc\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x18WIDATx\x9c\xed\xddw\\\x13\xf7\xff\xc0\xf1\xcf\xe52%\t+\t\x08\x01\x14gU\x14,*u#Z7\x82\xfa\xc5\x85h\x1du\xa0_m\xc5\xf5m\x1d\xa5Z\\\xb8g\xebjk)\xb6\x15\xab\xd6j\xdd\xbb\xa0\xa8(\xb8q\x01\x01\xd9\x04\x12\x12\x92\x0b\xfc\xfe\x88?\x14\x0c\xc1\x87\x86\xdc}\xe4\xfd\xfc\x8b\xde\xe5\x927\xc9\xab\xe7%$\x17\xa2\xa2\xa2\x02\x01\x803\x16\xdd\x03\x00\xf0\xbe b\x80=\x88\x18`\x0f"\x06\xd8\x83\x88\x01\xf6 b\x80=v\xad\x97P<\xd6\xe4g\xe94%\x06\xab\xcc\xc3\\\\\x01Kh\xcb\x96\xc9y\xb6R\x0e\xdd\xb3\xd4N]L\xbdx\xa6-)\xa4\xcaJ\xcb\xe9\x9e\xe5\xdd\tD\xa4\xa4!\xd7\xa5\x89\xc0\xfc\xc5\x083\xaf\x13\xeb\xb4\xe5\x7fn\xcf\xe4pY\xb6\x12.\x97_\xdf\xf7\xd9<\x01\x99\x9d\xa6!\x10rr\xe7\xf9\xf6\xb1\xa7{\x1cs\x92/+\x9f\xa6\x94\x12$rr\x17\xe8\xb4\x18G\\\xa6)/\xc9\xd7QT\xf9\x90).\x1c^\x8d\x05\xd6\x18\xb1N[~\xe4\xfb,\x9f\x00G\xa9\x9c_\x97s\xe2\xe7\xca\xe1\x1cg\x0f^\xbb\xee\xb6

In [12]:
initial_state = {
    'messages' : [
        HumanMessage(
            content="엔코아 고등학교, 엔코어 대학교, 봉사활동 100시간, 토익 900점, 경제 동아리 활동"
        )
    ]
}

In [13]:
for ouput in graph.stream(initial_state):
    message_type = 'generate' if 'generate' in ouput else 'reflect'
    print("\nNew Message: ", ouput[message_type]['messages'][-1].content, ".....")


New Message:  자기소개서

데이터에 근거한 사고와 꾸준함, 그리고 봉사로 다져진 책임감을 바탕으로 성장하는 인재입니다. 엔코아 고등학교와 엔코어 대학교를 거치며 기본에 충실하고, 과정을 끝까지 완수하는 태도를 가장 큰 장점으로 키워 왔습니다.

학부 시절에는 경제 동아리 활동을 통해 시장 이슈를 구조적으로 바라보는 시각을 길렀습니다. 시사·산업 자료를 분석하고 의견을 정리해 공유하는 과정을 반복하며 숫자와 사실에 기반해 논리를 세우는 연습을 꾸준히 이어갔습니다. 이 경험은 문제를 빠르게 단정하지 않고, 데이터를 확인해 가설을 검증하는 업무 태도로 이어졌습니다.

봉사활동 100시간을 통해서는 공감과 실행의 균형을 배웠습니다. 다양한 연령대의 사람들과 협업하며 상황에 맞는 소통 방식과 역할 분담의 중요성을 체감했고, 약속한 일을 끝까지 책임지는 신뢰의 가치를 몸으로 익혔습니다. 이는 팀워크가 요구되는 어떤 환경에서도 안정적으로 기여할 수 있는 자산이라고 생각합니다.

또한 TOEIC 900점의 영어 역량을 바탕으로 기본적인 문서 읽기, 메일 커뮤니케이션, 자료 탐색을 원활히 수행할 수 있습니다. 국내외 자료를 가리지 않고 정보를 확보·정리해 의사결정에 연결하는 데 강점을 갖고 있습니다.

앞으로는 경제적 관점과 데이터 기반 사고, 봉사로 다진 실행력을 바탕으로 조직의 목표에 기여하겠습니다. 주어진 문제를 구조화하고, 사실과 수치로 대안을 제시하며, 끝까지 완수하는 사람으로 성장하겠습니다. 감사합니다.

원하는 직무/산업, 기업명(또는 공공/대외활동), 강조하고 싶은 경험이 있다면 알려주시면 해당 용도에 맞춰 더 정밀하게 다듬어 드리겠습니다. .....

New Message:  다음은 인사 담당자 관점에서의 비평과 구체 수정 제안입니다.

총평
- 강점: 일관된 톤, 데이터·책임감·협업 역량을 핵심 메시지로 일관되게 전달. 문장 구조가 안정적이며 가독성이 좋음.
- 보완점: 사례가 추상적임. 역할·행동·결과(수치/영향)가 부족해 차별성이 약함. 목표